# Image Classification with CNN

In this project we will build a convolutional neural network (CNN) to classify images from the [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html).

# Image Dataset

The [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) consists of 60000 32x32 colour images in 10 categories - airplanes, dogs, cats, and other objects. The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class. 

In the following we will preprocess the images, then train a convolutional neural network on all the samples. The images need to be normalized and the labels need to be one-hot encoded.  Next we will build a convolutional, max pooling, dropout, and fully connected layers. At the end, we will train the network ang get to see it's predictions on the sample images.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.metrics import Precision, Recall
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import seaborn as sns
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.datasets import cifar10

### 1. Data Preprocessing
In this section, we will load the CIFAR-10 dataset, normalize the pixel values, and prepare it for training.

1.1 Load and splitting the dataset

In [ ]:
# Load CIFAR-10 dataset and split between testing and training data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

1.2 Checking the Size of the Data


We'll now check the shapes of the training and testing data to understand the dataset structure.


In [ ]:
# Check shapes of the dataset
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

1.3 Visualize the data

In [ ]:
for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(x_train[i], cmap=plt.get_cmap('gray'))
plt.show()

1.4 Normalizing the Data

We will normalize the pixel values of the images by scaling them between 0 and 1.


In [5]:
# Normalize pixel values between 0 and 1
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Convert labels to categorical (one-hot encoding)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

1.5 Data augmentation: starting with a small rotation range between the standard so the images don't get distorted

In [6]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)

### 2. Model Definition

2.1 Defining the CNN Model


We will define a CNN model with three convolutional layers followed by max-pooling and dense layers for classification.

2.2 F1-Score Custom Metric

In [22]:
# Define the custom F1-Score metric as a class
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Update the precision and recall
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        # Compute the F1 score using the precision and recall
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

1.3 Model Parameters


Here, we set key parameters for the model such as the input shape and the number of classes.

In [23]:
input_shape = (32, 32, 3)
num_classes = 10

In [24]:
def create_cnn_model():
# Define the VGG-style model
  model = Sequential([
    # Block 1
    tf.keras.Input(shape=input_shape),
    Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'),
    Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    # Block 2
    Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'),
    Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    # Block 3
    Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'),
    Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    # Block 4
    Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'),
    Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'),
    Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    # Block 5
    Conv2D(512, kernel_size=(3, 3), padding='same', activation='relu'),
    Conv2D(512, kernel_size=(3, 3), padding='same', activation='relu'),
    Conv2D(512, kernel_size=(3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    # Fully Connected Layers
    Flatten(),
    Dropout(0.5),  # Dropout to reduce overfitting
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),  # L2 regularization
    Dense(num_classes, activation='softmax')  # Final classification layer
  ])
  return model

# Instantiate the model
model = create_cnn_model()

2.4 Model summary 

print out the model summary to visualize the architecture, the number of layers, and the parameters.

In [ ]:
model.summary()

### 3. Model Compilation

3.1 Learning Rate Scheduler


We will define a custom learning rate scheduler that reduces the learning rate by 10% every 10 epochs.

In [26]:
# Custom learning rate scheduler
def scheduler(epoch, lr):
    if epoch % 10 == 0 and epoch != 0:
        lr = lr * 0.9
    return lr

# Create the LearningRateScheduler callback
lr_scheduler = LearningRateScheduler(scheduler)

3.2 Model Compliation


We will compile the model using Adam optimizer and categorical cross-entropy as the loss function, tracking accuracy as a metric.


In [29]:
f1_metric = F1Score()

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall(), f1_metric])

### 4. Model Training

We will now train the model for 25 epochs with a batch size of 512, using the learning rate scheduler callback to adjust the learning rate dynamically during training.


In [ ]:
# Train the model
history = model.fit(x_train, y_train, 
                    batch_size=512, 
                    epochs=25, 
                    callbacks=[lr_scheduler])

#### 5. Model Evaluation
We will evaluate the model on the test data and calculate the accuracy.


In [ ]:
# Evaluate the model on the test data
test_results = model.evaluate(x_test, y_test, verbose=0)
print(f'Test Loss: {test_results[0]}')
print(f'Test Accuracy: {test_results[1]}')

5.1 Bar Plot for Training Metrics

In [ ]:
train_loss, train_acc, train_precision, train_recall, train_f1 = model.evaluate(x_train, y_train)

print('Train loss:', train_loss)
print('Train accuracy:', train_acc)
print('Train precision:', train_precision)
print('Train recall:', train_recall)
print('Train F1-score:', train_f1)

test_loss, test_acc, test_precision, test_recall, test_f1 = model.evaluate(x_test, y_test)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)
print('Test precision:', test_precision)
print('Test recall:', test_recall)
print('Test F1-score:', test_f1)

# Bar Plot for Training Data Metrics
train_metrics = ['Loss', 'Accuracy', 'Precision', 'Recall', 'F1-Score']
train_values = [train_loss, train_acc, train_precision, train_recall, train_f1]

plt.figure(figsize=(10, 5))
plt.bar(train_metrics, train_values, color=['blue', 'orange', 'green', 'red', 'purple'])
plt.ylim(0, 1)  # Set y-axis limit to [0, 1] for better visibility
plt.title('Model Evaluation Metrics on Training Data')
plt.ylabel('Value')
plt.xlabel('Metrics')
plt.grid(axis='y')
plt.show()

# Subplots for Training History Metrics
plt.figure(figsize=(12, 12))  # Adjust the figure size

# Cross Entropy Loss
plt.subplot(411)  # First subplot (Cross Entropy Loss)
plt.plot(history.history['loss'], color='blue', label='Train Loss')
plt.title('Cross Entropy Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend()

# Accuracy
plt.subplot(412)  # Second subplot (Accuracy)
plt.plot(history.history['accuracy'], color='green', label='Train Accuracy')
plt.title('Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend()

# Precision and Recall
plt.subplot(413)  # Third subplot (Precision)
plt.plot(history.history['precision'], color='orange', label='Precision')
plt.plot(history.history['recall'], color='red', label='Recall')
plt.title('Precision and Recall')
plt.ylabel('Value')
plt.xlabel('Epochs')
plt.legend()

# F1 Score
plt.subplot(414)  # Fourth subplot (F1 Score)
plt.plot(history.history['f1_score'], color='purple', label='F1-Score')
plt.title('F1-Score')
plt.ylabel('F1-Score')
plt.xlabel('Epochs')
plt.legend()

plt.tight_layout()  # Avoids overlap between subplots
print("Training Data Metrics:")
plt.show()


metrics = ['Loss', 'Accuracy', 'Precision', 'Recall', 'F1-Score']
values = [test_loss, test_acc, test_precision, test_recall, test_f1]

plt.figure(figsize=(10, 5))
plt.bar(metrics, values, color=['blue', 'orange', 'green', 'red', 'purple'])
plt.ylim(0, 1)  # Set y-axis limit to [0, 1] for better visibility
plt.title('Model Evaluation Metrics on Test Data')
plt.ylabel('Value')
plt.xlabel('Metrics')
plt.grid(axis='y')
print("Test Data Metrics:")

plt.show()

# Subplots for Training History Metrics
plt.figure(figsize=(12, 12))  # Adjust the figure size

# Cross Entropy Loss
plt.subplot(411)  # First subplot (Cross Entropy Loss)
plt.plot(history.history['loss'], color='blue', label='Train Loss')
plt.title('Cross Entropy Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend()

# Accuracy
plt.subplot(412)  # Second subplot (Accuracy)
plt.plot(history.history['accuracy'], color='green', label='Train Accuracy')
plt.title('Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend()

# Precision and Recall
plt.subplot(413)  # Third subplot (Precision)
plt.plot(history.history['precision'], color='orange', label='Precision')
plt.plot(history.history['recall'], color='red', label='Recall')
plt.title('Precision and Recall')
plt.ylabel('Value')
plt.xlabel('Epochs')
plt.legend()

# F1 Score
plt.subplot(414)  # Fourth subplot (F1 Score)
plt.plot(history.history['f1_score'], color='purple', label='F1-Score')
plt.title('F1-Score')
plt.ylabel('F1-Score')
plt.xlabel('Epochs')
plt.legend()

plt.tight_layout()  # Avoids overlap between subplots
plt.show()

5.2 Generate confusion matrix

In [ ]:
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=range(10), yticklabels=range(10))
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

## Transfer Learning
In this section, we will apply transfer learning using a pre-trained model ResNet50 and fine-tune it for our classification task.


In [ ]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Set the desired image size for ResNet50 (224x224)
IMG_SIZE = 224
BATCH_SIZE = 32

# Function to resize images
def resize_images(image, label):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    return image, label

# Create a tf.data dataset and resize images dynamically
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.map(resize_images).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.map(resize_images).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

# Load ResNet50 feature extractor from TensorFlow Hub
feature_extractor_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"
feature_extractor_layer = hub.KerasLayer(feature_extractor_url, input_shape=(IMG_SIZE, IMG_SIZE, 3), trainable=False)

# Build your transfer learning model
model = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Dense(10, activation='softmax')
    ])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model on CIFAR-10
history = model.fit(train_dataset,
                    epochs=10)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_dataset)
print(f'Test Accuracy: {test_acc}')